In [1]:
#import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import sqlite3

# Fix the query so it pulls years sequentially and then drops duplicates


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\rdel1csa\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\rdel1csa\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\rdel1csa\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\rdel1csa\Anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\rdel1csa\Anaconda

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\rdel1csa\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\rdel1csa\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\rdel1csa\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\rdel1csa\Anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\rdel1csa\Anaconda

AttributeError: _ARRAY_API not found

In [2]:
# Read in the cluster assignments
nums = pd.read_csv('HC_clustering.csv', header=None)
nums = nums.drop(0, axis=1)
nums_vector = nums.to_numpy().reshape(-1,1)
nums_vector = nums_vector[~np.isnan(nums_vector)]

# Read resolve ecoregions data.
shapefile_path = r"C:\Users\rdel1csa\Documents\TreeMap\resolve ecoregions\Biomes_and_Ecoregions_2017.shp"
polygons = gpd.GeoDataFrame.from_file(shapefile_path)
AR_poly = polygons[polygons['ECO_NAME'] == 'Ozark Mountain forests']

# Assign the cluster IDs to the polygons
polygons['cluster'] = nums_vector

# Read in the forest data from the FIDDB
#conn = sqlite3.connect(r'C:\Users\rdel1csa\Documents\TreeMap\SQLite_FIADB_AR.db') # test run using only Arkansas
conn = sqlite3.connect(r'C:/Users/rdel1csa/Documents/TreeMap/SQLite_FIADB_ENTIRE.db')

# Run a for loop to gather older years first, followed by recent years, so we can drop duplicates later
dfs = []
for i in range(1900, 2024):
    query = 'select LAT, LON, FLDTYPCD, plot.INVYR, plot.cn from plot join cond on plot.CN = COND.PLT_CN where FLDTYPCD is not null and plot.INVYR = ' +str(i)+';'
    df = pd.read_sql_query(query, conn)
    if df.empty == False:
        dfs.append(df)

# Compile the queries into a single dataframe and convert to a geodataframe
finished = pd.concat(dfs)
points = gpd.GeoDataFrame(
    finished, 
    geometry=gpd.points_from_xy(finished.LON, finished.LAT),
    crs="EPSG:4326"
)

# Drop the duplicate entries, for plots measured multiple years
points.drop_duplicates(subset = ['geometry'], keep = 'last', inplace = True)

C:\Users\rdel1csa\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:196: RuntimeWarning: C:\Users\rdel1csa\Documents\TreeMap\resolve ecoregions\Biomes_and_Ecoregions_2017.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [4]:
# Short script to create a dataframe with the Foresst Types (a link to FLDTYPCD)
table = pd.read_csv('forest_codes.txt')
table.columns = ['head']
table = table[table['head'] != '-']
table = table[table['head'] != 'W']
table = table[table['head'] != 'E']
num = []
forest = []
for i, row in enumerate(table['head']):
    if i % 2 == 0:
        num.append(int(row))
    else:
        forest.append(row)
forest_codes = pd.DataFrame(columns=['forest code', 'forest name'])
forest_codes['forest code'] = num
forest_codes['forest name'] = forest

In [13]:
polygons['cluster'].unique()[76:]

array([ 77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101.])

In [7]:
# Run a loop over all shapefiles and locate the points within

# Make a copy because I'm going to drop points as I
# assign them to polys, to speed up subsequent search.
pts = points.copy()
pts.reset_index(drop=True, inplace = True)

# We're going to keep a list of how many points we find.
pts_in_polys = []

# Loop over all clusters created by the clustering algorithm.
start = 76 # This number helps us reset if need be. Skipped 77 because it wasnt moving
for cluster in polygons['cluster'].unique()[start:]:

    print(cluster)

    # Polygons belonging to a specific cluster
    cluster_polys = polygons[polygons['cluster'] == cluster]
    
    # Keep a list of points in this poly
    pts_in_this_cluster = []

    for i, poly in cluster_polys.iterrows():
        # Now loop over all points with index j.
        for j, pt in pts.iterrows():
            if poly.geometry.contains(pt.geometry):
                # Then it's a hit! Add it to the list,
                # and drop it so we have less hunting.
                pts_in_this_cluster.append(pt.geometry)
                pts = pts.drop([j])
                print('point:',j)
    print('points left:', pts.shape[0])

    # Add the number of points for each poly to the dataframe.
    gpd_series = gpd.GeoSeries(pts_in_this_cluster, name = 'geometry')

    eco_points = gpd.pd.merge(points, gpd_series, left_on = 'geometry', right_on = 'geometry')

    forest_counts = eco_points['FLDTYPCD'].value_counts()
    forest_counts.index = [int(x) for x in forest_counts.index]
    sum_of_trees = forest_counts.sum()
    forest_counts = forest_counts.apply(lambda x: (x/sum_of_trees)*100)
    forest_counts = forest_counts.round(1)

    # Merge the counted forest types with the forest codes to prepare the report for this ecoregion
    ecoregion_result = pd.merge(forest_counts, forest_codes, left_index= True, right_on = 'forest code')

    # Add to the result information on the ecoregions examined
    ecoregion_result = pd.concat([ecoregion_result,pd.Series(cluster_polys['BIOME_NAME'].unique())],axis=1)
    ecoregion_result = pd.concat([ecoregion_result,pd.Series(cluster_polys['REALM'].unique())],axis=1)
    ecoregion_result = pd.concat([ecoregion_result,pd.Series(cluster_polys['ECO_NAME'].unique())],axis=1)

    # Set all the column names
    ecoregion_result.columns = ['forest percent', 'forest code', 'forest type', 'cluster biomes', 'cluster realms', 'cluster ecoregions']
    
    # write the result
    fid = 'dump/Cluster '+ str(cluster) +'.csv'
    ecoregion_result.to_csv(fid, index = False)

# There were about 33,000 points left when  restarted this

77.0


KeyboardInterrupt: 